In [2]:
from SimCsstLens.LensPop.Population import LensPopulation
from SimCsstLens import CosmologyDistance as CD
import math
import h5py
import numpy as np 
from multiprocessing import Pool
import os
from astropy.table import Table, vstack

this_cosmos = CD.CosmosDist(Om0=0.3, Ode0=0.7, h=0.7)
lens_pop = LensPopulation(
    vdisp_floor=50, 
    zl_max=2.5, 
    cosmo_dist=this_cosmos,
    src_catalog_type='lsst',
    bands=['g', 'r', 'i', 'z'],
)
sky_frac = 17500.0/41252.96
N_etgs = lens_pop.dfl_pop.number_of_etgs(sky_frac=sky_frac) #ideal lenses
Nsamples_per_draw = 500000
Ndraw = math.ceil(N_etgs/Nsamples_per_draw)

In [3]:
import h5py
import numpy as np 
from multiprocessing import Pool
import os

current_dir = os.getcwd()
def return_detected(count_draw, stack=False):
    fn = h5py.File(f"{current_dir}/samples/lenses_{count_draw}.hdf5", "r")
    src_z = fn['source/z'][()]
    src_Re = fn['source/Re'][()]
    src_q = fn['source/q'][()]
    src_pa = fn['source/pa'][()]
    src_app_mag_g = fn['source/app_mag_g'][()]
    src_app_mag_r = fn['source/app_mag_r'][()]
    src_app_mag_i = fn['source/app_mag_i'][()]
    src_thetaE = fn['source/thetaE'][()]

    obs_cond = fn['Obs/detect_cond'][()] #shape: [nsrc, nband, n_ideal_lenses, 3]
    SNR = fn['Obs/SNR'][()] #shape: [nsrc, nband, n_ideal_lens]
    mu_arr = fn['Obs/magnification'][()] #shape: [nsrc, nband, n_ideal_lens]
    ring_cond = fn['Obs/ring_cond'][()] #shape: [nsrc, nband, n_ideal_lenses]

    fn.close()

    n_ideal_lens = src_thetaE.shape[1]

    # #src 0 and 1
    if stack:
        #stack band
        this_obs_cond = obs_cond.astype('int') #src, band-stack, nlens, 3-condition
        this_obs_cond = this_obs_cond[:, -1, :, :] #only use the stack band
        this_obs_cond = np.sum(this_obs_cond, axis=2) #shape: [nsrc, nlens]
        bool_cond = (this_obs_cond==3) #shape: [nsrc, nlens]
        bool_cond = (np.sum(bool_cond.astype('int'), axis=0)==2) #shape: [nlens]
    else:
        #individual band
        this_obs_cond = obs_cond.astype('int')
        this_obs_cond = this_obs_cond[:, :-1, :, :] #remove stack band
        this_obs_cond = np.sum(this_obs_cond, axis=3) #shape: [nsrc , nbands, nlens]
        bool_cond = (this_obs_cond==3) #shape: [nsrc , nbands, nlens]
        bool_cond = (np.sum(bool_cond.astype('int'), axis=0)==2) #shape: [nbands, nlens]
        bool_cond = np.sum(bool_cond.astype('int'), axis=0)>0 #shape: [nlens]
        
#     print(SNR.shape)
#     print(SNR[0, :, bool_cond].shape)

    n_detect_lens = len(np.where(bool_cond)[0])
    return n_detect_lens, n_ideal_lens, SNR[0, :, bool_cond], mu_arr[0, -1, bool_cond], ring_cond[0, -1, bool_cond], obs_cond

n_detect_lens, n_ideal_lens, snr, mu, ring_cond, obs_cond = return_detected(0, stack=True)

In [4]:
print(n_detect_lens, n_ideal_lens)

0 73


In [5]:
pool = Pool(processes=24)
vars = list(zip(list(range(Ndraw)), [True]*Ndraw))
results = pool.starmap(return_detected, vars)

n_detect_lens = [item[0] for item in results]
n_ideal_lens = [item[1] for item in results]
snr = [item[2] for item in results]
mu = [item[3] for item in results]
ring_cond = [item[4] for item in results]
obs_cond = [item[5] for item in results]

n_detect_lens = sum(n_detect_lens)
n_ideal_lens = sum(n_ideal_lens)
snr_stack_false = np.concatenate(snr, axis=0)
mu = np.concatenate(mu, axis=0)
ring_cond = np.concatenate(ring_cond, axis=0)
obs_cond = np.concatenate(obs_cond, axis=2)

etg_lensing_rate = n_ideal_lens / N_etgs
detect_rate_of_ideal_lens = n_detect_lens / n_ideal_lens
ring_lenses_rate = len(np.where(ring_cond)[0])/n_detect_lens
N_non_resolved_lenses = len(np.where(~obs_cond[0, -1, :, 0])[0])
N_non_tangent_arc_lenses = len(np.where(~obs_cond[0, -1, :, 1])[0])
N_lack_snr_lenses = len(np.where(~obs_cond[0, -1, :, 2])[0])

print('----------rate----------------')
print(f'ideal lensing rate of early type galaxy: {etg_lensing_rate:.2%}, == {n_ideal_lens}/{N_etgs}')
print(f'ideal lenses detectable rate: {detect_rate_of_ideal_lens:.2%}')
print(f'rate of a lens is ring-like: {ring_lenses_rate:.2%}')
print(f'{(N_non_resolved_lenses/n_ideal_lens):.2%} of ideal lenses are not detectable partially due to *resolved image condition*')
print(f'{(N_non_tangent_arc_lenses/n_ideal_lens):.2%} of ideal lenses are not detectable partially due to *tangential arc condition*')
print(f'{(N_lack_snr_lenses/n_ideal_lens):.2%} of ideal lenses are not detectable partially due to *lacking enough SNR*')
print('------------------------------')
print()

from SimCsstLens.SimLensImage.MockSurvey import MockSurvey
survey = MockSurvey(config_path=f'.', config_file='csst_setting.yaml')
factor = survey.survey_sky_frac/sky_frac
observed_lens_survey = n_detect_lens*factor
print('----------lenses number----------------')
print("n lenses whole survey", observed_lens_survey)
print("n ring-like lenses whole survey", observed_lens_survey*ring_lenses_rate)
print('------------------------------')

----------rate----------------
ideal lensing rate of early type galaxy: 0.01%, == 113675/899037691.0961848
ideal lenses detectable rate: 0.13%
rate of a lens is ring-like: 38.51%
14.31% of ideal lenses are not detectable partially due to *resolved image condition*
57.44% of ideal lenses are not detectable partially due to *tangential arc condition*
93.18% of ideal lenses are not detectable partially due to *lacking enough SNR*
------------------------------

----------lenses number----------------
n lenses whole survey 147.99999551755687
n ring-like lenses whole survey 56.99999827365366
------------------------------


In [11]:
"""
mask0 = stacked_table['if_obs_stack0'].data
table_detect0 = stacked_table[mask0]

mask1 = stacked_table['if_obs_stack1'].data
table_detect1 = stacked_table[mask1]

mask_one = (stacked_table['if_obs_stack0'].data | stacked_table['if_obs_stack1'].data)
table_detect_one = stacked_table[mask_one]

mask_two = (stacked_table['if_obs_stack0'].data & stacked_table['if_obs_stack1'].data)
table_detect_two = stacked_table[mask_two]

print(len(table_detect0), len(table_detect1), len(table_detect_one), len(table_detect_two))
"""

4384 4293 8529 148


In [13]:
table_detect_two

thetaE_s0,mass_s0,z_s0,re_s0,q_s0,pa_s0,x_s0,y_s0,mag_g_s0,mag_r_s0,mag_i_s0,mag_z_s0,SNR_g0,SNR_r0,SNR_i0,SNR_z0,SNR_stack0,best_band0,mu0,if_ring0,if_obs_stack0,if_obs_single0,thetaE_s1,mass_s1,z_s1,re_s1,q_s1,pa_s1,x_s1,y_s1,mag_g_s1,mag_r_s1,mag_i_s1,mag_z_s1,SNR_g1,SNR_r1,SNR_i1,SNR_z1,SNR_stack1,best_band1,mu1,if_ring1,if_obs_stack1,if_obs_single1,vdisp_l,re_l,q_l,z_l,mag_g_l,mag_r_l,mag_i_l,mag_z_l,noise_seed
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,bool,bool,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,bool,bool,bool,float64,float64,float64,float64,float64,float64,float64,float64,int64
1.0197665942543468,482066993197.0103,1.1425,0.1680661134517104,0.7180466100822196,55.06615266608102,-0.1785185502260828,-0.2695422081125793,25.7121,25.1161,24.7512,24.1066,12.563789077337956,16.13737176265859,16.9185926478917,19.278290634708878,32.58487539217718,z,6.877790696972994,True,True,False,0.677691491673668,320360268268.24963,0.8923,0.2615256607794205,0.526711380791733,132.58706362521164,0.1903277729647986,-0.5563294993205314,23.6139,23.2062,22.525,22.1234,37.83780766203311,40.71184989509612,55.84049371543675,51.65893751888107,93.21156029398053,i,3.1346298954286724,False,True,True,308.24015442037313,0.7181244862710964,0.6508934616880845,0.6251051370148925,22.130224682710733,20.218490878702752,18.829265000135003,18.241501806527246,41
1.599132578852834,506354736733.8201,2.034,0.045646056191220565,0.44028061863017565,83.66888909422329,-0.1273995894775506,-1.3277777279671719,27.17,27.2756,27.1903,26.9684,15.924568293290886,10.982304872246807,9.341652003518126,7.33493324073846,21.675119719523636,g,10.26628136041342,True,True,False,1.5774739158557014,499496664585.8641,1.9256,0.024027296990594034,0.6622021608733757,32.426275613869024,0.1062540964435713,1.0798396428714152,26.8744,26.9018,26.8563,26.6146,19.293889379362778,13.977076902193142,11.292119782809493,8.754112705584976,26.52312202974295,g,7.4319067823346465,True,True,False,282.04855227675466,0.6258051970311556,0.23287250877082422,0.40942678532295507,20.568063105229744,18.63049988562655,17.88328609057151,17.425055142225233,3
0.9787916956505031,232954397852.24228,2.2374,0.15944204242707624,0.41089374460859895,87.71948079948501,0.06442160552804843,-0.4678960761535167,24.9348,24.9591,25.0216,24.9595,23.112574335027137,17.682929403087062,13.368271039485732,9.262838963915923,31.624789051323013,g,5.420996745552729,False,True,True,0.9806187793986718,233389247469.65485,2.252,0.041723096480049265,0.7481824376709152,16.675077102345796,0.10329945747830319,0.2977483203802469,26.4803,26.5486,26.6255,26.6623,19.242073293176297,14.099438071893985,10.38572232639351,6.456578464271218,25.247749785075122,g,6.271573216499264,False,True,False,229.03457340022374,0.5492102688972984,0.9815060268835816,0.5167900660378532,22.59046773681445,20.74764629389524,19.611437754495302,19.07921240054356,16
1.226467664278539,230329621884.80832,1.9885,0.056238733719484665,0.795244525506495,157.73557969096908,0.04008236009473929,-0.19469460649170026,26.6525,26.703,26.7854,26.7383,25.230381167688375,17.245194199502535,12.397895737487556,8.364405491545511,31.17155500103302,g,18.29989995996854,True,True,True,1.0883826986555465,204397378543.82327,1.2219,0.08350611522767896,0.705145570028817,162.01484883365757,0.3705001258632565,-0.40426412776269716,24.824,24.8874,24.8188,24.3273,33.578777939215996,23.8188347615846,19.964081977414796,20.00130713918043,48.11962965053701,g,3.687992412752993,False,True,True,236.67800210062947,0.6232199919338075,0.7563348820828112,0.31338588429402725,20.41383492809969,18.568986746981178,17.929533264065316,17.58729561345277,24
1.088078860426324,253793074429.81952,1.7337,0.25967956092127115,0.8189641005880531,15.607814262904649,0.019525065899577296,0.10240200926801817,25.5676,25.7011

In [5]:
mask_thetaE = (stacked_table['thetaE_s0'].data > 0.074)
stacked_table = stacked_table[mask_thetaE]
stacked_table.write('lens_samples.fits', overwrite=True)
stacked_table[0]

thetaE_s0,mass_s0,z_s0,re_s0,q_s0,pa_s0,x_s0,y_s0,mag_g_s0,mag_r_s0,mag_i_s0,mag_z_s0,vdisp_l,re_l,q_l,z_l,mag_g_l,mag_r_l,mag_i_l,mag_z_l,SNR_g,SNR_r,SNR_i,SNR_z,SNR_stack,best_band,mu,if_ring,if_obs_stack,if_obs_single,noise_seed
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,bool,bool,bool,int64
1.0966084016507394,220618705190.45496,2.4813,0.04755680836013041,0.48724151839551133,56.84835836540002,-0.679774694569647,-0.19032010755702378,26.8804,26.7911,27.1784,26.9373,227.31589216009394,0.48874105992050526,0.6177436725840784,0.39480433148639105,21.44529815513444,19.494334310672166,18.780101509455797,18.338227104944824,8.565924589820026,6.563163395814959,3.5898829774557766,2.7892841517608145,10.657003960170975,g,2.8410611440345908,False,False,False,0


In [6]:
len(stacked_table)

5186409

In [9]:
mask = stacked_table['if_obs_stack'].data
table_detect = stacked_table[mask]
table_detect.write("catalog_csv/csst_wf_stack.csv", overwrite=True)
len(table_detect)

161548

In [10]:
mask = stacked_table['if_obs_single'].data
table_detect = stacked_table[mask]
table_detect.write("catalog_csv/csst_wf_single.csv", overwrite=True)
len(table_detect)

76396